In [1]:
import sys
import os
import pandas as pd
import json

# Add the project root directory to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

# Now you can import the IntentClassifier from src/infer_intent.py
from src.infer_intent import IntentClassifier
from tqdm import tqdm
import pandas as pd

In [2]:
# with open('../data/internal/search_examples.txt', 'r', encoding='utf-8') as f:
#     yelp_texts = f.read().split('\n')

In [3]:
# yelp_texts[:10]

## Using yelp Keywords for validation

In [4]:
tmp2 = pd.read_json("https://firefox-settings-attachments.cdn.mozilla.net/main-workspace/quicksuggest/33987d71-9e87-4b7e-86d3-6f292b89e8bf.json")
yelp_texts = tmp2['subjects'][0]

In [5]:
len(tmp2['subjects'][0])

2870

In [6]:
# !ls -ltrsh models/

In [7]:
cls = IntentClassifier()

#### Yelp internal data evaluation (python)

In [8]:
yelp_hit_count = 0
yelp_queries_misclassified = []
for query in tqdm(yelp_texts):
    pred_result, _ = cls.find_intent(query)
    if pred_result == 'yelp_intent':
        yelp_hit_count += 1
    else:
        yelp_queries_misclassified.append({'query': query, 'pred_result': pred_result})
print(f"yelp hit rate (accuracy) = {yelp_hit_count/len(yelp_texts)}")

  1%|█▍                                                                                                                                                                                                                                                             | 16/2870 [00:00<00:42, 67.57it/s]/Users/cgopal/work/smart_intent/src/infer_intent.py:52: RuntimeWarning: overflow encountered in exp
  probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
/Users/cgopal/work/smart_intent/src/infer_intent.py:52: RuntimeWarning: invalid value encountered in divide
  probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2870/2870 [00:42<00:00, 67.36it/s]

yelp hit rate (accuracy) = 0.8742160278745644


In [9]:
## old
# yelp hit rate (accuracy) = 0.3256021409455843

## new (Mozilla/mobilebert-uncased-finetuned-LoRA-intent-classifier)
# yelp hit rate (accuracy) = 0.6703835860838537
# yelp hit rate (accuracy) = 0.7832292595896521
# yelp hit rate (accuracy) = 0.8862622658340767
# yelp hit rate (accuracy) = 0.8742160278745644

In [10]:
yelp_queries_misclassified_df = pd.DataFrame(yelp_queries_misclassified)
print(f"Number of yelp queries misclassified = {len(yelp_queries_misclassified_df)} out of {len(yelp_texts)} examples")
yelp_queries_misclassified_df

Number of yelp queries misclassified = 361 out of 2870 examples


,query,pred_result
0,24 hour maid service,navigation_intent
1,aaliyah beauty and brows,information_intent
2,absolute bagels,information_intent
3,ahgassi gopchang,information_intent
4,aldi,information_intent
...,...,...
356,yangmani,information_intent
357,ye's apothecary,information_intent
358,yoga,information_intent
359,you move me,information_intent


#### Load the Fx ML inference output (using Fx js)

In [11]:
fx_ml_intent = pd.read_json("../data/ML_output_YELP_KEYWORDS_DATA.json")
fx_ml_hit_rate_for_yelp_intent = float(fx_ml_intent['intent'].value_counts(normalize=True)['yelp_intent'])
print(f"Fx ML yelp intent hit rate = {fx_ml_hit_rate_for_yelp_intent}")

Fx ML yelp intent hit rate = 0.8735191637630662


## Using Yelp additional validation dataset

In [12]:
tqdm.pandas()

In [13]:
yelp_val_generated_data = pd.read_csv("../data/yelp_val_generated_data.csv")
yelp_val_generated_data['pred_result'] = yelp_val_generated_data['queries'].progress_apply(lambda query: cls.find_intent(query)[0])
hit_rate = float(yelp_val_generated_data['pred_result'].value_counts(normalize=True)['yelp_intent'])
print(f"yelp generated val data hit rate (accuracy) = {hit_rate}")

 17%|███████████████████████████████████████████                                                                                                                                                                                                                   | 339/2000 [00:05<00:24, 67.14it/s]/Users/cgopal/work/smart_intent/src/infer_intent.py:52: RuntimeWarning: overflow encountered in exp
  probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
/Users/cgopal/work/smart_intent/src/infer_intent.py:52: RuntimeWarning: invalid value encountered in divide
  probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:30<00:00, 64.68it/s]

yelp generated val data hit rate (accuracy) = 0.982


In [14]:
# yelp generated val data hit rate (accuracy) = 0.982

In [15]:
queries_array = yelp_val_generated_data['queries'].values

# Create a dictionary structure
data = {
    "queries": queries_array.tolist()  
}

# Define the output JSON file path
output_file_path = "../data/yelp_val_generated_data.json"

## Uncomment to write it else its almost static dataset
# # Write to a JSON file
# with open(output_file_path, "w") as json_file:
#     json.dump(data, json_file, indent=2)


In [16]:
## yelp validation from Fx MLSuggest

fx_ml_intent_val = pd.read_json("../data/ML_output_YELP_VAL_DATA.json")
fx_ml_hit_rate_for_yelp_intent_2 = float(fx_ml_intent_val['intent'].value_counts(normalize=True)['yelp_intent'])
print(f"Fx ML yelp intent2 hit rate = {fx_ml_hit_rate_for_yelp_intent_2}")

Fx ML yelp intent2 hit rate = 0.9835


#### Weather internal data evaluation

In [17]:
with open('../data/internal/weather_search_examples.txt', 'r', encoding='utf-8') as f:
    weather_texts = f.read().split('\n')

In [18]:
weather_hit_count = 0
weather_queries_misclassified = []
for query in tqdm(weather_texts):
    pred_result, _ = cls.find_intent(query)
    if pred_result == 'weather_intent':
        weather_hit_count += 1
    else:
        weather_queries_misclassified.append({'query': query, 'pred_result': pred_result})
print(f"weather hit rate (accuracy) = {weather_hit_count/len(weather_texts)}")

 17%|████████████████████████████████████████████                                                                                                                                                                                                                      | 7/41 [00:00<00:00, 69.24it/s]/Users/cgopal/work/smart_intent/src/infer_intent.py:52: RuntimeWarning: overflow encountered in exp
  probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
/Users/cgopal/work/smart_intent/src/infer_intent.py:52: RuntimeWarning: invalid value encountered in divide
  probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [00:00<00:00, 65.44it/s]

weather hit rate (accuracy) = 0.6585365853658537


In [19]:
## old
# weather hit rate (accuracy) = 0.3902439024390244

## new (Mozilla/mobilebert-uncased-finetuned-LoRA-intent-classifier)
# weather hit rate (accuracy) = 0.4878048780487805
# weather hit rate (accuracy) = 0.6341463414634146
# weather hit rate (accuracy) = 0.6585365853658537

In [20]:
weather_queries_misclassified_df = pd.DataFrame(weather_queries_misclassified)
print(f"Number of weather queries misclassified = {len(weather_queries_misclassified)} out of {len(weather_texts)} examples")
weather_queries_misclassified_df

Number of weather queries misclassified = 14 out of 41 examples


,query,pred_result
0,forecast,information_intent
1,flooding,yelp_intent
2,forcast,information_intent
3,wather,yelp_intent
4,vindy,yelp_intent
5,cloudy,information_intent
6,air quality,yelp_intent
7,pollen,information_intent
8,tiempo,information_intent
9,clima,information_intent


#### Weather validate the generated data with pre and post modifiers

In [21]:
weather_val_generated_data = pd.read_csv("../data/weather_val_generated_data.csv")
weather_val_generated_data['pred_result'] = weather_val_generated_data['queries'].progress_apply(lambda query: cls.find_intent(query)[0])
hit_rate = float(weather_val_generated_data['pred_result'].value_counts(normalize=True)['weather_intent'])
print(f"weather generated val data hit rate (accuracy) = {hit_rate}")

  0%|                                                                                                                                                                                                                                                                        | 0/2000 [00:00<?, ?it/s]/Users/cgopal/work/smart_intent/src/infer_intent.py:52: RuntimeWarning: overflow encountered in exp
  probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
/Users/cgopal/work/smart_intent/src/infer_intent.py:52: RuntimeWarning: invalid value encountered in divide
  probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:30<00:00, 66.14it/s]

weather generated val data hit rate (accuracy) = 0.643


In [22]:
# weather generated val data hit rate (accuracy) = 0.643

#### Orcas golden dataset for intent validation

In [23]:
orcas_data = pd.read_csv("../data/ORCAS_golden_dataset.tsv", sep='\t')
print(len(orcas_data))

1000


In [24]:
orcas_data.head()

,query,url,label_manual,target
0,best reads,http://thegreatestbooks.org/,Abstain,information_intent
1,tamerind,https://en.wikipedia.org/wiki/Tamarind,Factual,information_intent
2,show mi ip,http://showip.net/,Instrumental,navigation_intent
3,do carpenter ants eat wood,https://doyourownpestcontrol.com/carp.htm,Factual,information_intent
4,rheumatoid arthritis in children,https://www.webmd.com/rheumatoid-arthritis/und...,Abstain,information_intent


In [25]:
orcas_data['target'].value_counts()

target
information_intent    591
navigation_intent     265
unknown                78
purchase_intent        27
yelp_intent            15
weather_intent         14
travel_intent           9
translation_intent      1
Name: count, dtype: int64

In [26]:
# orcas_data['query'].values

intent_hit_count = 0
intent_queries_misclassified = []
intent_queries_classified = []
for query, target in tqdm(orcas_data[['query', 'target']].values.tolist()):
    pred_result, _ = cls.find_intent(query)
    intent_queries_classified.append({'query': query, 'pred_result': pred_result, 'target': target})
    if pred_result == target:
        intent_hit_count += 1
    else:
        intent_queries_misclassified.append({'query': query, 'pred_result': pred_result, 'target': target})
print(f"intent hit rate (accuracy) = {intent_hit_count/len(orcas_data)}")

  0%|                                                                                                                                                                                                                                                                        | 0/1000 [00:00<?, ?it/s]/Users/cgopal/work/smart_intent/src/infer_intent.py:52: RuntimeWarning: overflow encountered in exp
  probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
/Users/cgopal/work/smart_intent/src/infer_intent.py:52: RuntimeWarning: invalid value encountered in divide
  probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:15<00:00, 65.68it/s]

intent hit rate (accuracy) = 0.521


In [27]:
# intent hit rate (accuracy) = 0.521

In [28]:
intent_queries_classified_df = pd.DataFrame(intent_queries_classified)
intent_queries_misclassified_df = pd.DataFrame(intent_queries_misclassified)
# intent_queries_misclassified_df

In [29]:
intent_queries_misclassified_df['target'].value_counts()

target
information_intent    205
navigation_intent     160
unknown                78
purchase_intent        25
travel_intent           9
yelp_intent             2
Name: count, dtype: int64

In [30]:
pd.crosstab(intent_queries_classified_df['target'], intent_queries_classified_df['pred_result'])

pred_result,information_intent,navigation_intent,purchase_intent,translation_intent,weather_intent,yelp_intent
target,,,,,,
information_intent,386,28,5,1,8,163
navigation_intent,66,105,0,0,2,92
purchase_intent,8,2,2,0,0,15
translation_intent,0,0,0,1,0,0
travel_intent,3,1,0,0,1,4
unknown,49,10,0,0,0,19
weather_intent,0,0,0,0,14,0
yelp_intent,2,0,0,0,0,13


In [31]:
intent_queries_misclassified_df.loc[intent_queries_misclassified_df['target'] == 'yelp_intent']

,query,pred_result,target
283,gordo's,information_intent,yelp_intent
391,orthopedic spine specialist,information_intent,yelp_intent


In [32]:
from sklearn.metrics import classification_report

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [33]:
print(classification_report(intent_queries_classified_df['target'],
                            intent_queries_classified_df['pred_result']))

                    precision    recall  f1-score   support

information_intent       0.75      0.65      0.70       591
 navigation_intent       0.72      0.40      0.51       265
   purchase_intent       0.29      0.07      0.12        27
translation_intent       0.50      1.00      0.67         1
     travel_intent       0.00      0.00      0.00         9
           unknown       0.00      0.00      0.00        78
    weather_intent       0.56      1.00      0.72        14
       yelp_intent       0.04      0.87      0.08        15

          accuracy                           0.52      1000
         macro avg       0.36      0.50      0.35      1000
      weighted avg       0.65      0.52      0.56      1000



/Users/cgopal/work/smart_intent/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/cgopal/work/smart_intent/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/cgopal/work/smart_intent/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap

In [34]:
# intent_queries_misclassified_df.loc[intent_queries_misclassified_df['pred_result'] == 'yelp_intent']['target'].value_counts()